In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
### Reding csv file containing 243 subjects and their raw TPM counts across 1059 genes
### Source of data
myDF = pd.read_excel("C:/Users/filip/OneDrive/Desktop/Molecular Stethoscope/Summer/ClusterMarkers_1819ADcohort.xlsx", sheet_name = 1)
# setting index row name to the gene id
myDF = myDF.set_index('gene_id')

#Filtering out rows: discarding the ERCC rows, ERCC is a control protocol for validation of RNA sequencing
Patients_df = myDF[~myDF.loc[:,'Coeff'].isnull()]

# We store the coefficients(betas) of the linear classifier in an array.
coefficients = np.nan_to_num(np.array(Patients_df.loc[:, "Coeff"]))

# Filtering out columns with patient data
Patients_df = Patients_df.filter(regex='^\d+')

# group columns by patient id
grouped_cols = Patients_df.columns.str.split('-').str[0]

# group columns by patient id and r1/r2 suffixes
grouped = Patients_df.groupby(grouped_cols, axis=1)

# apply the mean function to the r1 and r2 columns for each group
# taking mean of the replicates for subjects with multiple replicates
Patients_df = grouped.apply(lambda x: x.mean(axis=1)).reset_index(drop=True)
# Patients_df.head()

Patients_df['Mean']= Patients_df.mean(axis=1)
Patients_df['Std']=Patients_df.iloc[:,:-1].std(axis=1)
Patients_df['RSD'] = (Patients_df['Std'] / Patients_df['Mean']) # New code Filip
Patients_df.head()
#print(Patients_df.shape)

# We define a function whose input is TPM and outputs the corresponding Zscore
def z_score(x):
    return (x-x['Mean'])/x['Std']

# Computing and storing zscores
Patients_df_zScore = Patients_df.apply(lambda x: z_score(x), axis=1)
Patients_df_zScore.head()
#print(Patients_df_zScore.shape)

patient_id= list(Patients_df_zScore.columns.values)
patient_id = patient_id[0:243]
#patient_id

#np.random.seed(46215423)

# Sampling function performing the Monte Carlo simulations
def Simulation(means, std, coefficients):
    return np.sum(np.multiply(coefficients, np.random.normal(means, std, size=(1, len(coefficients)))))

# Function to perform anti-logit operation on the linear score 
def cl_score(linear_score, gamma = 0):
    temp = gamma + linear_score
    classifier_score = np.exp(temp) / (1 + np.exp(temp))
    return classifier_score

# Function to calculate subject wise mean and std of simulated scores 
def run_sim_one_patient_mean_sd(col,percent):
    std = [percent/100 * val for val in col]
    std = np.abs(std)
    temp_Sim = [Simulation(col, std, coefficients) for _ in range(numRuns)]
    return [np.mean(temp_Sim),np.std(temp_Sim)]

# Function to calculate the classifier score for each simulation of "num_runs" simulations, corresponding to each subject
def run_sim_one_patient(col, percent, num_runs):
    std = [percent/100 * val for val in col]
    std = np.abs(std)
    temp_Sim = np.asarray([cl_score(Simulation(col, std, coefficients)) for _ in range(num_runs)])
    return temp_Sim

# This score is the classifer linear score we want to compare with the simulated scores
def linear_score(coefficients, col):
    linear_score = np.sum(coefficients * col, axis=0)
    return linear_score

num_runs = 1000
uncertainty = 25
thresh = 0.04874941

def plot_uncert_at_thresh(num_runs , uncertainty, thresh ):
    
    
    """The purpose of this function is to produce 2 things based on %RSD (uncertainty), threshold (thresh), number of MC simulations (num_runs) :
    1. Figure to show the scattering based on classifier scores of simulations against subject.
    2. False Positves/Negatives
    
    pseudocode:
    1.  for a range of %RSD, the code starts with the first %RSD.
    2a. for each subject, it calculates the linear score by multiplying the z-scores present in dataframe "Patients_df_zScore"  by coefficient. Function used: linear_score
    2b. then it performs anti-logit operation on the linear score to get the classifier score.  Function used: cl_score 
    3.  similarly, "scores" stores the classifier score for each simulation of "num_runs"simulations, corresponding to each subject. Function used: run_sim_one_patient
    4a. since, we will plot the simulation scores against the subject scores, x_data creates the same array shape for subject scores as simulation scores.
    4b. y_data is just scores for the simplicity.
    5.  Now coming to each simuation score in each subject: we will sort out the simulation as FP, FN, TP, TN based on threshold using if-else condtions. this code 
        will run for num_runs.
    6.  based on FP, FN, TP, TN we will calculate the accuracy.
    7.  the process 2-6 will be repeated for each subject.
    8.  A scatter plot will be produced using simulation scores on y-axis and subject scores on x-axis, two perpendicular lines passing the thresholds on x and y axis
     will  categorize the scatter dots into FP, FN , TP and TN.
    
    
    Example input: sub_accuracy(400, 50, 0.87)
    """
    
    false_pos = 0 # counter, stores number of points on second quadrant
    false_neg = 0
    num_subj_unreliable = np.zeros(243) # keeps track of subjects whose score is unreliable under the assumed variation (0 is fine, 1 is unreliable)
    fig = plt.figure(figsize=(10,10))
    #for i in range(243):
    for i in range(243):
        scores = run_sim_one_patient(Patients_df_zScore.iloc[:, i], uncertainty, num_runs)
        y_0 = cl_score(linear_score(coefficients, Patients_df_zScore.iloc[:, i]))
        x_data = np.ones_like(scores) * y_0
        y_data = scores
        colour = np.zeros_like(x_data)
        for j in range(len(x_data)):
            if x_data[j] > thresh and y_data[j] < thresh:
                colour[j] = 1
                false_neg = false_neg + 1
                num_subj_unreliable[i] = 1
            elif x_data[j] < thresh and y_data[j]> thresh:
                colour[j] = 2
                false_pos = false_pos + 1
                num_subj_unreliable[i] = 1
        plt.scatter(x_data, y_data, c=colour, cmap = 'Dark2_r' ,alpha = 0.15, s=100)
    plt.axvline(x = thresh, color = 'g', linestyle= '--')
    plt.axhline(y = thresh, color = 'g', linestyle= '--')
    #plt.xlim([0.425, 0.575])
    #plt.ylim([0.425, 0.575])
    plt.xlabel("Classifier score", fontsize= 24)
    plt.ylabel("Simulated scores", fontsize = 24)
    plt.title('Uncertainty around threshold', fontsize = 28)
    # Below are the labels of agreement and disagreement
    A = plt.scatter([0],[0], alpha =0) # dummy plot for legend
    B = plt.scatter([0],[0],  alpha =0) # dummy plot for legend
    plt.text(0, 0.8, 'A', fontsize = 20)
    plt.text(0, 0, 'B', fontsize =20)
    plt.text(0.8, 0, 'B', fontsize =20)
    plt.text(0.8, 0.8, 'A', fontsize =20)
    plt.legend([A,B], ['A : Agreement', 'B : Disagreement'])
    #plt.savefig('figures/uncert_around_thresh_25_RSD.png')

    return [ fig , false_pos , false_neg , (np.sum(num_subj_unreliable)) ]

cl_score(-2.97108439) # checking the threshold value in probability

### Check that random seeding is working by running this cell several times... it is working.
np.random.seed(2)
scores = run_sim_one_patient(Patients_df_zScore.iloc[:, 1], uncertainty, num_runs)
print(scores)
len(scores)

# Example for using the above function
plot_uncert_at_thresh(100,50,0.86)
plot_uncert_at_thresh(num_runs,uncertainty,thresh)

def sub_accuracy(uncertainty_range, thresh, num_runs):
    """The purpose of this function is to calculate 2 things based on %RSD (uncertainty_range), threshold (thresh), number of MC simulations (num_runs) :
    1. subject accuracy based on number of simulations : this info is provided in a dataframe called "accuracy_df"
    2. True Positives/Negatives, False Positves/Negatives: this info is provided in a dtatframe called "false_pos_df"
    
    pseudocode:
    1.  for a range of %RSD, the code starts with the first %RSD.
    2a. for each subject, it calculates the linear score by multiplying the z-scores present in dataframe "Patients_df_zScore"  by coefficient. Function used: linear_score
    2b. then it performs anti-logit operation on the linear score to get the classifier score.  Function used: cl_score 
    3.  similarly, "scores" stores the classifier score for each simulation of "num_runs"simulations, corresponding to each subject. Function used: run_sim_one_patient
    4a. since, we will plot the simulation scores against the subject scores, x_data creates the same array shape for subject scores as simulation scores.
    4b. y_data is just scores for the simplicity.
    5.  Now coming to each simuation score in each subject: we will sort out the simulation as FP, FN, TP, TN based on threshold using if-else condtions. this code will run for num_runs.
    6.  based on FP, FN, TP, TN we will calculate the accuracy.
    7.  the process 2-6 will be repeated for each subject.
    8.  the process 2-7 will be repeated for each %RSD input. 
    
    
    Example input: sub_accuracy([20, 40, 50], 0.87, 400)  -> note that uncertainty_range should be given in a form of a list even if giving single value.
    """
    global real_score
    global accuracy_df
    global y_data
    global x_data
    global Sub_score 
    global false_pos_df
    global fig
   
    np.random.seed(46215423)
    
    real_score = np.zeros(243)
    accuracy = np.zeros(243)
    
    false_pos_df = pd.DataFrame(columns = uncertainty_range, index= ['simulations in agreement with subject', 'simulations in disagreement with subject', 'Unreliable Subjects'])
    
    num_subj_unreliable = np.zeros(243) # keeps track of subjects whose score is unreliable under the assumed variation (0 is fine, 1 is unreliable)
    false_pos_series=[]
   
    AD=0
    NCI=0
    Sub_score = pd.Series(real_score)
    accuracy_df= pd.DataFrame(columns= uncertainty_range)
    
    
    for i in range(len(uncertainty_range)):
        false_pos = 0 # counter, stores number of points on second quadrant
        false_neg = 0
        true_pos = 0
        true_neg = 0
        FN = []
        FP = []
        TN = []
        TP = []
        
        for j in range(243):
            scores = run_sim_one_patient(Patients_df_zScore.iloc[:, j], uncertainty_range[i], num_runs)
            y_0 = cl_score(linear_score(coefficients, Patients_df_zScore.iloc[:, j]))
            x_data = np.ones_like(scores) * y_0
            y_data = scores
            colour = np.zeros_like(x_data)
            false_neg = 0
            false_pos = 0
            true_neg=0
            true_pos=0
            for k in range(len(x_data)):
                if x_data[k] > thresh and y_data[k] < thresh:
                    false_neg = false_neg + 1
                    num_subj_unreliable[j] = 1
                elif x_data[k] < thresh and y_data[k]> thresh:
                    false_pos = false_pos + 1
                    num_subj_unreliable[j] = 1
                elif x_data[k]> thresh and y_data[k] > thresh:
                    true_pos = true_pos +1
                elif x_data[k] < thresh and y_data[k] < thresh:
                    true_neg = true_neg +1  

            accuracy[j] = (num_runs-(false_neg)-(false_pos))/num_runs
            real_score[j] = y_0


            FN.append(false_neg)
            FP.append(false_pos)
            TN.append(true_neg)
            TP.append(true_pos)
        unreliable_subjects= str(np.sum(num_subj_unreliable))
        accuracy_series = pd.Series(accuracy)
        
        disagreement = sum(FN) + sum(FP)
        agreement= sum(TN) + sum(TP)
       
        accuracy_df[accuracy_df.columns[i]] = accuracy_series*100
        false_pos_df[false_pos_df.columns[i]]= [agreement, disagreement, unreliable_subjects]
        Sub_score = pd.Series(real_score)    
        accuracy_df['Classifier Score']= Sub_score
        
        
        dfm = accuracy_df.melt('Classifier Score', var_name='%RSD', value_name='Agreement')   
    fig = sns.relplot(x= "Classifier Score" , y="Agreement", hue='%RSD', data=dfm, kind='line', linewidth= 1, palette = "rainbow")
    accuracy_df['Patient ID'] = patient_id
    accuracy_df = accuracy_df.set_index('Patient ID')
    for l in range (len(Sub_score)):
            if Sub_score[l]>thresh:
                AD= AD+1
            else: NCI =NCI+1
    return ( print("AD=",AD), print("NCI=",NCI), accuracy_df, false_pos_df, fig)


sub_accuracy([25],thresh,100)

accuracy_df

disagreement_df=  accuracy_df[accuracy_df[25] < 100]
disagreement_df = disagreement_df.sort_values(by= [25], ascending= False)
disagreement_df.shape

disagreement_df.to_csv("subject-wise disagreement.csv")

false_pos_df

accuracy_df

sub_accuracy([10, 25, 50], 0.86, 100)
sub_accuracy([10, 25, 50], 0.90, 100)
sub_accuracy([10, 25, 50], 0.80, 100)




In [ ]:
'''
1. Import necessary libraries: pandas, numpy, matplotlib, and seaborn.

2. Load a CSV file containing 243 subjects and their raw TPM counts across 1059 genes.

3. Set the index row name to the gene id.

4. Filter out rows that do not contain ERCC (a control protocol for validation of RNA sequencing).

5. Store the coefficients (betas) of the linear classifier in an array.

6. Filter out columns with patient data.

7. Group columns by patient id and r1/r2 suffixes.

8. Apply the mean function to the r1 and r2 columns for each group.

9. Calculate the mean, standard deviation, and relative standard deviation (RSD) of each row.

10. Define a function to compute the Z-score of each row.

11. Compute and store the Z-scores.

12. Define a sampling function performing the Monte Carlo simulations.

13. Define a function to perform anti-logit operation on the linear score.

14. Define a function to calculate subject wise mean and standard deviation of simulated scores.

15. Define a function to calculate the classifier score for each simulation, corresponding to each subject.

16. Define a function to calculate the linear score.

17. Define a function to produce a scatter plot based on classifier scores of simulations against subject and count false positives and negatives.

18. Define a function to calculate subject accuracy based on the number of simulations and produce dataframes with the accuracy and false positive/negative data.

19. Calculate and output the subject accuracy for a given uncertainty range, threshold, and number of Monte Carlo simulations.

20. Save the results to a CSV file.
'''